## Part 2: Business Questions

In [0]:
# Import required libraries
import pyspark.sql.functions as F
from pyspark.sql.functions import col, unix_timestamp, round

In [0]:
# Read the saved table
spark.sql("DROP TABLE IF EXISTS taxi_database.taxi_table")
spark.sql("DROP DATABASE IF EXISTS taxi_database")
spark.sql("CREATE DATABASE taxi_database")
spark.sql("USE taxi_database")
spark.sql("CREATE TABLE taxi_table USING PARQUET OPTIONS (path '/dbfs/mnt_1/taxi_combined.parquet')")

Out[2]: DataFrame[]

#### 1. Business Question 1

In [0]:
result1 = spark.sql("""
                    WITH total_trips AS 
                    (
                        SELECT 
                            YEAR(lpep_pickup_datetime) AS year,
                            MONTH(lpep_pickup_datetime) AS month,
                            CONCAT(YEAR(lpep_pickup_datetime), '-', LPAD(MONTH(lpep_pickup_datetime), 2, '0')) AS year_month,
                            COUNT(*) AS total_trips,
                            ROUND(AVG(passenger_count), 0) AS average_passenger_count,
                            ROUND(AVG(total_amount), 2) AS average_total_amount,
                            ROUND(AVG(total_amount / passenger_count), 2) AS average_amount_per_passenger
                        FROM taxi_table
                        WHERE passenger_count > 0
                        GROUP BY YEAR(lpep_pickup_datetime), MONTH(lpep_pickup_datetime)
                ),
                    day_of_week_ AS (
                        SELECT 
                            YEAR(lpep_pickup_datetime) AS year,
                            MONTH(lpep_pickup_datetime) AS month,
                            CONCAT(YEAR(lpep_pickup_datetime), '-', LPAD(MONTH(lpep_pickup_datetime), 2, '0')) AS year_month,
                            date_format(lpep_pickup_datetime, 'EEEE') AS day_of_week,
                            COUNT(*) AS total_trips,
                            RANK() OVER (PARTITION BY YEAR(lpep_pickup_datetime), MONTH(lpep_pickup_datetime) ORDER BY COUNT(*) DESC) AS rank
                        FROM taxi_table
                        GROUP BY YEAR(lpep_pickup_datetime), MONTH(lpep_pickup_datetime), date_format(lpep_pickup_datetime, 'EEEE')
                    ),
                    hour_of_day AS (
                        SELECT 
                            YEAR(lpep_pickup_datetime) AS year,
                            MONTH(lpep_pickup_datetime) AS month,
                            CONCAT(YEAR(lpep_pickup_datetime), '-', LPAD(MONTH(lpep_pickup_datetime), 2, '0')) AS year_month,
                            date_format(lpep_pickup_datetime, 'ha') AS hour_,
                            COUNT(*) AS total_trips,
                            RANK() OVER (PARTITION BY YEAR(lpep_pickup_datetime), MONTH(lpep_pickup_datetime) ORDER BY COUNT(*) DESC) AS rank
                        FROM taxi_table
                        GROUP BY YEAR(lpep_pickup_datetime), MONTH(lpep_pickup_datetime), date_format(lpep_pickup_datetime, 'ha')
                    )
                    SELECT 
                        t.year_month,
                        t.total_trips,
                        d.day_of_week AS most_trips_day_of_week,
                        h.hour_ AS most_trips_hour_of_day,
                        t.average_passenger_count,
                        t.average_total_amount,
                        t.average_amount_per_passenger
                    FROM total_trips AS t
                    JOIN day_of_week_ AS d 
                        ON t.year = d.year AND t.month = d.month AND d.rank = 1
                    JOIN hour_of_day AS h 
                        ON t.year = h.year AND t.month = h.month AND h.rank = 1
                    ORDER BY t.year, t.month
                    """)

In [0]:
result1.display()

year_month total_trips most_trips_day_of_week most_trips_hour_of_day average_passenger_count average_total_amount average_amount_per_passenger 2015-01 8875952 Saturday 7PM 2.0 17.72 14.44 2015-02 8753784 Saturday 7PM 2.0 18.5 15.14 2015-03 9572483 Sunday 9PM 2.0 18.92 15.47 2015-04 9530635 Thursday 9PM 2.0 19.03 15.5 2015-05 9853447 Saturday 9PM 2.0 19.47 15.8 2015-06 9086315 Tuesday 9PM 2.0 19.47 15.83 2015-07 8580684 Thursday 10PM 2.0 19.06 15.44 2015-08 8352999 Saturday 9PM 2.0 19.08 15.43 2015-09 8283875 Wednesday 9PM 2.0 19.51 15.86 2015-10 8996993 Saturday 7PM 2.0 19.71 16.05 2015-11 8232090 Sunday 7PM 2.0 19.43 15.84 2015-12 8298644 Thursday 9PM 2.0 19.5 15.82 2016-01 7712879 Friday 6PM 2.0 18.76 15.34 2016-02 8010301 Saturday 6PM 2.0 18.75 15.4 2016-03 8767659 Thursday 7PM 2.0 19.19 15.71 2016-04 8593257 Saturday 10PM 2.0 19.46 15.89 2016-05 8641116 Sunday 9PM 2.0 19.92 16.24 2016-06 8098559 Thursday 9PM 2.0 19.97 16.31 2016-07 7505731 Friday 9PM 2.0 19.58 15.93 2016-08 7178227 Wednesday 9PM 2.0 19.62 15.98 2016-09 7231679 Friday 8PM 2.0 20.29 16.61 2016-10 7713511 Saturday 7PM 2.0 19.89 16.26 2016-11 7028069 Wednesday 9PM 2.0 20.02 16.42 2016-12 7156950 Friday 9PM 2.0 19.67 16.03 2017-01 6481996 Sunday 6PM 2.0 18.76 15.44 2017-02 6225891 Saturday 6PM 2.0 18.89 15.56 2017-03 721988 Friday 6PM 1.0 17.14 15.4 2017-04 6940636 Saturday 6PM 2.0 19.71 16.17 2017-05 6959285 Wednesday 9PM 2.0 20.27 16.68 2017-06 6668661 Thursday 9PM 2.0 20.15 16.55 2017-07 5977046 Saturday 9PM 2.0 19.76 16.14 2017-08 5845309 Thursday 6PM 2.0 19.85 16.24 2017-09 6155073 Saturday 7PM 2.0 20.25 16.6 2017-10 6654679 Sunday 7PM 2.0 20.22 16.59 2017-11 6184361 Wednesday 6PM 2.0 20.17 16.57 2017-12 6240100 Friday 9PM 2.0 19.89 16.19 2018-01 5622983 Wednesday 6PM 2.0 19.11 15.75 2018-02 5525879 Friday 6PM 2.0 19.12 15.8 2018-03 6168755 Saturday 6PM 2.0 19.73 16.25 2018-04 6171484 Sunday 6PM 2.0 20.11 16.55 2018-05 6223558 Thursday 9PM 2.0 20.62 16.99 2018-06 5898928 Friday 9PM 2.0 20.48 16.86 2018-07 5294856 Tuesday 6PM 2.0 20.03 16.43 2018-08 5278334 Thursday 6PM 2.0 20.14 16.54 2018-09 5385534 Saturday 6PM 2.0 20.66 17.05 2018-10 5847648 Wednesday 6PM 2.0 20.89 17.3 2018-11 5278243 Friday 6PM 2.0 20.8 17.19 2018-12 5285214 Saturday 6PM 2.0 20.42 16.76 2019-01 4808585 Thursday 6PM 2.0 19.28 16.04 2019-02 4555764 Friday 6PM 2.0 22.17 18.4 2019-03 5155869 Friday 6PM 2.0 22.68 18.75 2019-04 4879076 Tuesday 6PM 2.0 22.63 18.61 2019-05 4970663 Thursday 6PM 2.0 23.13 19.04 2019-06 4600433 Saturday 6PM 2.0 23.16 19.05 2019-07 4132886 Wednesday 6PM 2.0 22.79 18.67 2019-08 4009481 Thursday 6PM 2.0 22.65 18.5 2019-09 4282532 Sunday 6PM 2.0 23.16 19.09 2019-10 4621708 Thursday 6PM 2.0 23.1 19.07 2019-11 4347021 Friday 6PM 2.0 22.73 18.76 2019-12 4324471 Tuesday 6PM 2.0 23.0 18.84 2020-01 3983441 Friday 6PM 2.0 21.59 18.0 2020-02 3921635 Saturday 6PM 2.0 21.72 18.16 2020-03 1908506 Tuesday 6PM 1.0 21.03 17.81 2020-04 141641 Wednesday 5PM 1.0 17.83 16.11 2020-05 191576 Friday 4PM 1.0 18.31 16.32 2020-06 328921 Tuesday 3PM 1.0 19.3 16.94 2020-07 472287 Thursday 5PM 1.0 19.31 16.77 2020-08 598019 Monday 3PM 1.0 19.44 16.72 2020-09 803347 Wednesday 5PM 1.0 19.26 16.54 2020-10 999994 Thursday 5PM 1.0 19.26 16.5 2020-11 892431 Monday 3PM 1.0 19.06 16.4 2020-12 843189 Tuesday 3PM 1.0 19.13 16.46 2021-01 790489 Friday 3PM 1.0 18.81 16.23 2021-02 799409 Friday 3PM 1.0 19.2 16.55 2021-03 1138612 Wednesday 5PM 1.0 19.35 16.64 2021-04 1315483 Friday 5PM 1.0 20.14 17.25 2021-05 1565302 Saturday 6PM 1.0 20.82 17.68 2021-06 1800753 Wednesday 6PM 1.0 21.61 18.2 2021-07 1800151 Thursday 6PM 2.0 21.85 18.2 2021-08 1780704 Tuesday 6PM 1.0 22.14 18.55 2021-09 1906840 Thursday 6PM 1.0 22.73 19.15 2021-10 2237219 Friday 6PM 1.0 22.65 19.09 2021-11 2229416 Tuesday 6PM 1.0 23.19 19.53 2021-12 2068951 Thursday 3PM 1.0 23.4 19.45 2022-01 1530979 Saturday 6PM 1.0 21.2 18.15 2022-02 1889663 Saturday 6PM 1.0 22.11 18.89 2022-03 2339346 Thursday 6PM 1.0 23.19 19.76 2022-04 2367084 Friday 6PM

In [0]:
# Find the most frequent day of the week
most_frequent_day = result1.groupBy("most_trips_day_of_week") \
                           .agg(F.count("*").alias("count")) \
                           .orderBy(F.desc("count")) \
                           .limit(3)

# Find the most frequent hour of the day
most_frequent_hour = result1.groupBy("most_trips_hour_of_day") \
                            .agg(F.count("*").alias("count")) \
                            .orderBy(F.desc("count")) \
                            .limit(3)

In [0]:
# Show the results
most_frequent_day.display()

most_trips_day_of_week count Friday 21 Thursday 21 Saturday 21

In [0]:
most_frequent_hour.display()

most_trips_hour_of_day count 6PM 52 9PM 19 7PM 8

#### 2. Business Question 2

In [0]:
result2 = spark.sql("""
                    SELECT
                    taxi_color,

                    -- trip duration minutes
                    ROUND(AVG(trip_duration_hours * 60), 2) AS average_duration,
                    ROUND(PERCENTILE_APPROX(trip_duration_hours * 60, 0.5), 2) AS median_duration,
                    ROUND(MIN(trip_duration_hours * 60), 2) AS min_duration,
                    ROUND(MAX(trip_duration_hours * 60), 2) AS max_duration,

                    -- trip distance
                    ROUND(AVG(trip_distance_km), 2) AS average_distance,
                    ROUND(PERCENTILE_APPROX((trip_distance_km), 0.5), 2) AS median_distance,
                    ROUND(MIN(trip_distance_km), 2) AS min_distance,
                    ROUND(MAX(trip_distance_km), 2) AS max_distance,

                    -- speed
                    ROUND(AVG(speed_kmph), 2) AS average_speed,
                    ROUND(PERCENTILE_APPROX(speed_kmph, 0.5), 2) AS median_speed,
                    ROUND(MIN(speed_kmph), 2) AS min_speed,
                    ROUND(MAX(speed_kmph), 2) AS max_speed

                    FROM taxi_table
                    GROUP BY taxi_color                   
                    """)

In [0]:
result2.display()

taxi_color average_duration median_duration min_duration max_duration average_distance median_distance min_distance max_distance average_speed median_speed min_speed max_speed green 17.63 14.4 2.4 149.4 6.33 4.51 2.0 74.99 21.59 19.63 0.82 56.0 yellow 18.38 15.0 2.4 149.4 6.42 4.02 2.0 74.99 20.33 18.17 0.81 56.0

#### 3. Business Question 3

In [0]:
result3 = spark.sql("""
                    SELECT 
                        taxi_color,
                        pickup_borough,
                        dropoff_borough,
                        date_format(lpep_pickup_datetime, 'MMMM') AS month_name, 
                        date_format(lpep_pickup_datetime, 'EEEE') AS day_of_week_name, 
                        date_format(lpep_pickup_datetime, 'ha') AS time_of_day,
                        COUNT(*) AS total_trips,
                        ROUND(AVG(trip_distance_km), 2) AS average_distance,
                        ROUND(AVG(total_amount), 2) AS average_amount_per_trip,
                        ROUND(SUM(total_amount), 2) AS total_amount_paid
                    FROM taxi_table
                    GROUP BY 
                        taxi_color, 
                        pickup_borough,
                        dropoff_borough,
                        date_format(lpep_pickup_datetime, 'MMMM'),
                        date_format(lpep_pickup_datetime, 'EEEE'),
                        date_format(lpep_pickup_datetime, 'ha') 
                    ORDER BY 
                        month_name, 
                        day_of_week_name  
                    """)

In [0]:
result3.display()

taxi_color pickup_borough dropoff_borough month_name day_of_week_name time_of_day total_trips average_distance average_amount_per_trip total_amount_paid yellow Queens Manhattan April Friday 10AM 13783 19.39 50.92 701782.22 yellow Queens Manhattan April Friday 9AM 12247 18.95 50.39 617082.38 yellow Manhattan Manhattan April Friday 9AM 212466 4.14 16.7 3547121.2 yellow Queens Queens April Friday 11AM 3379 10.21 24.13 81547.36 yellow Manhattan Brooklyn April Friday 4AM 5986 9.74 24.22 144975.26 yellow Manhattan Bronx April Friday 6AM 1060 11.36 26.17 27734.99 yellow Queens Queens April Friday 10AM 3671 10.29 24.2 88831.0 yellow Brooklyn Bronx April Friday 1AM 22 25.08 52.55 1156.2 yellow Bronx Brooklyn April Friday 8AM 20 24.81 53.02 1060.45 yellow Manhattan Manhattan April Friday 2AM 57309 4.85 14.94 856406.18 yellow Manhattan Bronx April Friday 1AM 2099 14.8 31.84 66832.93 yellow Queens Staten Island April Friday 6PM 33 41.79 91.44 3017.57 yellow Queens Queens April Friday 11PM 5263 9.38 22.45 118145.65 yellow Manhattan Manhattan April Friday 4PM 190747 3.98 17.67 3370817.25 yellow Queens Bronx April Friday 12PM 275 24.06 53.11 14606.29 yellow Queens Brooklyn April Friday 6PM 4221 20.27 46.92 198066.28 yellow Queens Queens April Friday 12PM 3450 10.87 25.76 88872.34 yellow Manhattan Bronx April Friday 4PM 1203 13.66 39.63 47680.88 yellow Bronx Bronx April Friday 5AM 93 6.37 17.69 1645.27 yellow Queens EWR April Friday 3PM 27 51.08 148.17 4000.46 yellow Bronx Bronx April Friday 7AM 192 6.5 20.0 3840.15 yellow Queens Brooklyn April Friday 9PM 4390 20.16 43.57 191276.8 yellow Brooklyn Bronx April Friday 7AM 29 27.98 60.26 1747.51 yellow Bronx Bronx April Friday 7PM 187 6.17 19.46 3639.78 yellow Staten Island Manhattan April Friday 12AM 5 38.15 74.12 370.58 yellow Manhattan Queens April Friday 3AM 6231 11.2 27.15 169178.97 yellow Manhattan Bronx April Friday 10PM 3432 14.75 33.61 115339.51 yellow Queens Bronx April Friday 9AM 339 21.48 48.12 16312.72 yellow Brooklyn Queens April Friday 3AM 207 9.13 22.3 4616.8 yellow Manhattan Bronx April Friday 12PM 1122 13.98 34.07 38227.68 yellow Manhattan Bronx April Friday 8AM 1249 12.31 30.49 38083.08 yellow Manhattan EWR April Friday 9AM 599 27.99 95.42 57154.18 yellow Brooklyn Brooklyn April Friday 12AM 5342 4.97 15.38 82158.98 yellow Queens Bronx April Friday 4AM 54 15.74 35.38 1910.4 yellow Brooklyn Manhattan April Friday 6PM 1318 7.99 26.17 34498.17 yellow Manhattan Bronx April Friday 5AM 666 13.36 28.56 19022.2 yellow Bronx Manhattan April Friday 6PM 169 9.56 28.47 4810.95 yellow Manhattan Brooklyn April Friday 5AM 2374 10.16 25.23 59902.8 yellow Queens Brooklyn April Friday 1AM 1044 18.09 38.75 40450.41 yellow Queens Brooklyn April Friday 7PM 3933 20.53 46.2 181700.23 yellow Manhattan Manhattan April Friday 6AM 97475 4.23 13.36 1302034.34 yellow Manhattan Queens April Friday 11PM 16066 10.99 31.42 504830.63 yellow Bronx Manhattan April Friday 2PM 199 11.27 30.78 6124.51 yellow Manhattan Bronx April Friday 11AM 1096 14.2 34.08 37352.84 yellow Queens Queens April Friday 4AM 1302 6.12 16.5 21482.01 yellow Brooklyn Brooklyn April Friday 2PM 1760 5.33 18.58 32702.26 yellow Bronx Bronx April Friday 6PM 204 5.79 19.09 3894.53 yellow Manhattan EWR April Friday 10PM 33 28.57 90.49 2986.32 yellow Brooklyn Manhattan April Friday 7AM 3261 9.51 25.98 84728.28 yellow Manhattan Staten Island April Friday 12AM 114 29.96 78.74 8975.89 yellow Queens Queens April Friday 2PM 4883 11.37 27.44 133975.18 yellow Manhattan Queens April Friday 5AM 9054 16.98 42.54 385137.07 yellow Bronx Brooklyn April Friday 2PM 26 30.23 66.84 1737.9 yellow Manhattan Manhattan April Friday 11AM 202458 4.09 16.91 3422627.52 yellow Queens Bronx April Friday 7AM 281 24.96 56.73 15942.34 yellow Manhattan Brooklyn April Friday 2PM 6187 10.57 31.66 195897.04 yellow Queens EWR April Friday 10AM 8 53.64 148.96 1191.68 yellow Bronx Manhattan April Friday 1AM 65 7.51 20.38 1324.92 yellow Queens Staten Island April Friday 7AM 17 43.76 95.

In [0]:
result3.count()

95480

In [0]:
# Group by pickup_borough and sum the total_trips, then order by the count in descending order
top_boroughs = result3.groupBy("pickup_borough") \
                      .agg(F.sum("total_trips").alias("total_trips_count")) \
                      .orderBy(F.desc("total_trips_count")) \
                      .limit(3)

In [0]:
# Show the top 3 boroughs with the highest number of trips
top_boroughs.display()

pickup_borough total_trips_count Manhattan 387927222 Queens 46609022 Brooklyn 23744421

#### 4. Percentage of trips where the driver received tips

In [0]:
percentage_tips = spark.sql("""
                            SELECT
                            ROUND((SUM(CASE WHEN tip_amount > 0 THEN 1 ELSE 0 END) / COUNT(*)) * 100, 2) AS percentage_tips
                            FROM taxi_table
                            """)

In [0]:
percentage_tips.display()

percentage_tips 65.82

#### 5. Percentage trips where drivers received tips more than $5

In [0]:

percent_trips = spark.sql("""
                          SELECT 
                          ROUND((SUM(CASE WHEN tip_amount >= 5 THEN 1 ELSE 0 END) / 
                                SUM(CASE WHEN tip_amount > 0 THEN 1 ELSE 0 END)) * 100, 2) AS percent_tips
                          FROM taxi_table
                          """)

In [0]:
percent_trips.display()

percent_tips 16.52

#### 6. Classifying into bins and finding average speed & distance per dollar

In [0]:
aggregated_bins1 = spark.sql("""
                            WITH duration_bins AS (
                                SELECT 
                                    CASE
                                        WHEN trip_duration_hours < 5/60 THEN 'under 5 minutes'
                                        WHEN trip_duration_hours >= 5/60 AND trip_duration_hours < 10/60 THEN '5 to 10 minutes'
                                        WHEN trip_duration_hours >= 10/60 AND trip_duration_hours < 20/60 THEN '10 to 20 minutes'
                                        WHEN trip_duration_hours >= 20/60 AND trip_duration_hours < 30/60 THEN '20 to 30 minutes'
                                        WHEN trip_duration_hours >= 30/60 AND trip_duration_hours < 60/60 THEN '30 minutes to 1 hour'
                                        WHEN trip_duration_hours >= 1 THEN 'Over 1 hour'
                                        ELSE 'Unknown'
                                    END AS trip_duration_bin,
                                    ROUND(AVG(speed_kmph), 2) AS average_speed_kmph,
                                    ROUND(AVG((trip_distance_km) / total_amount), 2) AS average_distance_per_dollar
                                FROM taxi_table
                                GROUP BY trip_duration_bin
                            )
                            SELECT
                                trip_duration_bin,
                                average_speed_kmph,
                                average_distance_per_dollar
                            FROM duration_bins
                            ORDER BY 
                                CASE 
                                    WHEN trip_duration_bin = 'under 5 minutes' THEN 1
                                    WHEN trip_duration_bin = '5 to 10 minutes' THEN 2
                                    WHEN trip_duration_bin = '10 to 20 minutes' THEN 3
                                    WHEN trip_duration_bin = '20 to 30 minutes' THEN 4
                                    WHEN trip_duration_bin = '30 minutes to 1 hour' THEN 5
                                    WHEN trip_duration_bin = 'Over 1 hour' THEN 6
                                    ELSE 7
                                END
                        """)


In [0]:
aggregated_bins1.display()

trip_duration_bin average_speed_kmph average_distance_per_dollar under 5 minutes 32.66 0.27 5 to 10 minutes 21.6 0.26 10 to 20 minutes 18.76 0.27 20 to 30 minutes 20.05 0.3 30 minutes to 1 hour 24.57 0.36 Over 1 hour 22.3 0.47

#### 7. Maximum income bin
The taxi driver will be advised to take up rides between the timeframe of **5 to 10 minutes**. This is because this has the lowest average distance driven per dollar earned, thereby maximising his/her profits.

## End of Notebook 3